In [107]:
import os
import cv2
import numpy as np
import os
from tqdm import tqdm
from IPython.display import clear_output
from spiral_tools.utils import path_sequence,create_yolo_dataset_format
import shutil

#### veri seti seç

In [108]:
ds_list = os.listdir("./datasets")
for i,ds in enumerate(ds_list):
    print(i,ds)
    
print("\nIndex girin: ")
d_i = int(input("> "))

if d_i not in range(len(ds_list)):
    print("\nuygun bir index girin")
else:
    
    dataset_name = ds_list[d_i]    
    print("\nseçilen veriseti:",dataset_name)
    
    root = f"./datasets/{ds_list[d_i]}/detect"
    print("\nroot: ",root)

    clear_output()
    print("Veri seti: " ,dataset_name)


Veri seti:  Aerial_Maritime


#### Yeni boyut seç

In [109]:
resize_shape = 0 # büyük kenarı oranları aynı tutarak küçültür eğer büyük kenar bu değerden küçükse değiştirilmeyecek

# Yeniden Boyutlandırma
print("Oranlar aynı tutularak büyük kenarlar girilen değere göre resim boyutu küçültülsün mü? (eğer değerden küçükse resim boyutu aynen kalır)")
print("0: hayır, boyut değeri: evet")
n = int(input(">"))
if n > 0 :
    resize_shape = n
    print("Yeniden boyulandırma: ",n)
else:
    print("Yeniden boyutlandırma yok")
    
# yeni ds oluştur
new_ds_name = f"{dataset_name}_resized"
create_yolo_dataset_format("./datasets",new_ds_name)

Oranlar aynı tutularak büyük kenarlar girilen değere göre resim boyutu küçültülsün mü? (eğer değerden küçükse resim boyutu aynen kalır)
0: hayır, boyut değeri: evet
>960
Yeniden boyulandırma:  960
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/train  --  ./datasets/Aerial_Maritime_resized/detect/labels/train
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/test  --  ./datasets/Aerial_Maritime_resized/detect/labels/test
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/val  --  ./datasets/Aerial_Maritime_resized/detect/labels/val


In [110]:
def get_new_shape(frame_width,frame_height,resize_shape):
    frame_shape = (frame_width,frame_height)
    
    max_value = np.max(frame_shape)
    max_value_idx = np.argmax(frame_shape)
    
    if max_value > resize_shape:
        if max_value_idx == 0:
            new_height = int(resize_shape * frame_height / frame_width)
            new_width = resize_shape
        elif max_value_idx == 1:
            new_width = int(resize_shape * frame_width / frame_height)
            new_height = resize_shape
        frame_shape = (new_width,new_height)
        
    return frame_shape

In [111]:
# resimleri yeniden boyutlandır ve taşı
images_path = path_sequence(root,"images")


for i,c in tqdm( enumerate(["train","test","val"])):
    images_file = path_sequence(images_path,c)
    print("işleniyor: ", images_file)
    image_names = os.listdir(images_file)
   
    for im_name in image_names:
        # frame
        original_image = cv2.imread(path_sequence(images_file,im_name))
        image = original_image.copy()
        new_shape = get_new_shape(frame_width=image.shape[1],frame_height=image.shape[0],resize_shape=resize_shape)
        new_image = cv2.resize(image,new_shape)
        im_path = path_sequence(".","datasets",new_ds_name,"detect","images",c,im_name)
        cv2.imwrite(im_path,new_image)
        
# etiketleri kopyala
labels_path = path_sequence(root,"labels")
os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","train"))
os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","test"))
os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","val"))
shutil.copytree(labels_path,path_sequence(".","datasets",new_ds_name,"detect","labels"))

0it [00:00, ?it/s]

işleniyor:  ./datasets/Aerial_Maritime/detect/images/train


2it [00:00,  2.55it/s]

işleniyor:  ./datasets/Aerial_Maritime/detect/images/test
işleniyor:  ./datasets/Aerial_Maritime/detect/images/val


3it [00:01,  2.74it/s]


'./datasets/Aerial_Maritime_resized/detect/labels'

### Tüm verisetlerine uygula

In [113]:
resize_shape = 0 # büyük kenarı oranları aynı tutarak küçültür eğer büyük kenar bu değerden küçükse değiştirilmeyecek

# Yeniden Boyutlandırma
print("Oranlar aynı tutularak büyük kenarlar girilen değere göre resim boyutu küçültülsün mü? (eğer değerden küçükse resim boyutu aynen kalır)")
print("0: hayır, boyut değeri: evet")
n = int(input(">"))
if n > 0 :
    resize_shape = n
    print("Yeniden boyulandırma: ",n)
    



    for ds_name in tqdm(os.listdir("./datasets")):
        # yeni ds oluştur
        new_ds_name = f"{ds_name}_resized"
        create_yolo_dataset_format("./datasets",new_ds_name)
        
        root = f"./datasets/{ds_name}/detect"

        # resimleri yeniden boyutlandır ve taşı
        images_path = path_sequence(root,"images")


        for i,c in tqdm( enumerate(["train","test","val"])):
            images_file = path_sequence(images_path,c)
            print("işleniyor: ", images_file)
            image_names = os.listdir(images_file)

            for im_name in image_names:
                # frame
                original_image = cv2.imread(path_sequence(images_file,im_name))
                image = original_image.copy()
                new_shape = get_new_shape(frame_width=image.shape[1],frame_height=image.shape[0],resize_shape=resize_shape)
                new_image = cv2.resize(image,new_shape)
                im_path = path_sequence(".","datasets",new_ds_name,"detect","images",c,im_name)
                cv2.imwrite(im_path,new_image)



        # etiketleri kopyala
        labels_path = path_sequence(root,"labels")
        os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","train"))
        os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","test"))
        os.removedirs(path_sequence(".","datasets",new_ds_name,"detect","labels","val"))
        shutil.copytree(labels_path,path_sequence(".","datasets",new_ds_name,"detect","labels"))
        clear_output()
else:
    print("Yeniden boyutlandırma yok")

Oranlar aynı tutularak büyük kenarlar girilen değere göre resim boyutu küçültülsün mü? (eğer değerden küçükse resim boyutu aynen kalır)
0: hayır, boyut değeri: evet
>960
Yeniden boyulandırma:  960
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/train  --  ./datasets/Aerial_Maritime_resized/detect/labels/train
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/test  --  ./datasets/Aerial_Maritime_resized/detect/labels/test
Oluşturuldu ./datasets/Aerial_Maritime_resized/detect/images/val  --  ./datasets/Aerial_Maritime_resized/detect/labels/val


0it [00:00, ?it/s]

işleniyor:  ./datasets/Aerial_Maritime/detect/images/train


2it [00:00,  2.61it/s]

işleniyor:  ./datasets/Aerial_Maritime/detect/images/test
işleniyor:  ./datasets/Aerial_Maritime/detect/images/val


...

1it [01:09, 69.09s/it]

işleniyor:  ./datasets/Visdrone_distil/detect/images/test


2it [01:30, 41.12s/it]

işleniyor:  ./datasets/Visdrone_distil/detect/images/val


3it [01:33, 31.25s/it]


#### uygulanan veri setlerinin eski ersiyonlarını logla


In [118]:
for ds_name in tqdm(os.listdir("./datasets")):
    if ds_name.endswith("resized"):
        print("resized")
    else:
        shutil.move(path_sequence(".","datasets",ds_name),path_sequence(".","datasets_old",ds_name))

100%|█████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 519.22it/s]

resized
resized
resized
resized
resized
resized
resized
resized
resized
resized
resized
resized


True